In [10]:
import os
from dotenv import load_dotenv

# 載入 .env 檔案中的環境變數
load_dotenv()

# 將 API key 設定為 os 變數
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")  # 或者可以使用 GOOGLE_API_KEY


In [ ]:
from langchain.tools import BaseTool
from music21 import stream, chord
from typing import TypedDict, Optional


class Music21RendererTool(BaseTool):
    name: str = "music21_renderer"
    description: str = "Renders chords into MIDI files using Music21."
    
    def _run(self, chords: list) -> str:
        s = stream.Stream()
        for c in chords:
            s.append(chord.Chord(c))
        s.write('midi', fp='output.mid')
        return 'output.mid'
    
class StructureGeneratorTool(BaseTool):
    name = "structure_generator"
    description = "Generates musical structure using GPT-4."
    
    def _run(self, user_input: str) -> list:
        # 假設調用 GPT-4 API
        structure = ["verse", "chorus", "verse"]  # 示例輸出
        return structure
    
class MusicTheoryValidatorTool(BaseTool):
    name = "music_theory_validator"
    description = "Validates chord progressions based on music theory."
    forbidden = ['V-IV', 'ii-VII']
    
    def _run(self, chords: list) -> dict:
        for i in range(len(chords)-1):
            transition = f"{chords[i]}-{chords[i+1]}"
            if transition in self.forbidden:
                return {"valid": False, "error": f"禁止进行: {transition}"}
        return {"valid": True}


class State(TypedDict):
    user_input: str
    structure: Optional[list]
    chords: Optional[list]
    validation_result: Optional[dict]
    rendered_midi: Optional[str]
    current_step: str

PydanticUserError: Field 'name' defined on a base class was overridden by a non-annotated attribute. All field definitions, including overrides, require a type annotation.

For further information visit https://errors.pydantic.dev/2.10/u/model-field-overridden

In [15]:
from langgraph.graph import StateGraph

def structure_node(state):
    tool = StructureGeneratorTool()
    state["structure"] = tool._run(state["user_input"])
    state["current_step"] = "generate_chords"
    return state
def validation_node(state):
    tool = MusicTheoryValidatorTool()
    state["validation_result"] = tool._run(state["chords"])
    return state

def validation_router(state):
    if state["validation_result"]["valid"]:
        return "render"
    else:
        state["current_step"] = "generate_chords"
        return "state_router"
workflow = StateGraph(initial_state=State(user_input="生成一首流行歌曲", current_step="design_framework"))

# 添加節點
workflow.add_node("conductor", lambda state: state)  # 簡化示例
workflow.add_node("state_router", lambda state: state)
workflow.add_node("structure_generator", structure_node)
workflow.add_node("spectral_expert", lambda state: {"chords": ["C", "G", "Am", "F"]})
workflow.add_node("music_theory_validator", validation_node)
workflow.add_node("music21_renderer", lambda state: {"rendered_midi": "output.mid"})

# 添加邊
workflow.add_edge("conductor", "state_router")
workflow.add_conditional_edges("state_router", lambda state: state["current_step"], {
    "design_framework": "structure_generator",
    "generate_chords": "spectral_expert",
    "validate": "music_theory_validator"
})
workflow.add_edge("structure_generator", "state_router")
workflow.add_edge("spectral_expert", "state_router")
workflow.add_conditional_edges("music_theory_validator", validation_router, {
    "render": "music21_renderer",
    "state_router": "state_router"
})

# 編譯並運行
app = workflow.compile()
final_state = app.run()

NameError: name 'State' is not defined